In [4]:
!pip install transformers torch tensorflow

In [17]:
!pip install tf-keras tensorflow

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 9.3 MB/s eta 0:00:00


In [12]:
import os
import pdfplumber
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import re
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from wordcloud import WordCloud
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from collections import Counter

In [13]:
# ---- 1. PDFからテキストを抽出 ----
def extract_text_from_pdfs(folder_path):
    pdf_texts = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.pdf'):
            with pdfplumber.open(os.path.join(folder_path, file_name)) as pdf:
                text = ' '.join([page.extract_text() for page in pdf.pages if page.extract_text()])
                if text.strip():
                    pdf_texts[file_name] = text
    return pdf_texts

In [27]:
def get_top_words(text, n=20):
    words = re.findall(r'\b[a-zA-Z]+\b', text.lower())  # 単語抽出（英字のみ）
    words = [word for word in words if word not in ENGLISH_STOP_WORDS]  # ストップワード除去
    counter = Counter(words)
    return [word for word, _ in counter.most_common(n)]  # 上位N単語

In [15]:
def extract_conclusion_length(text):
    match = re.search(r'conclusion|summary', text, re.IGNORECASE)  # 結論部分を検索
    if match:
        conclusion_text = text[match.start():]  # 結論部分以降を取得
        return len(conclusion_text)
    return 0  # 見つからなかった場合は0

In [18]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def generate_summary(text, max_length=150):
    summary = summarizer(text[:1024], max_length=max_length, min_length=50, do_sample=False)  # 1024文字以内
    return summary[0]['summary_text']

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

C:\Users\sk062\miniconda3\envs\Study\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sk062\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [19]:
folder_path = r"C:\Users\sk062\OneDrive\デスクトップ\資料\Note info"
pdf_texts = extract_text_from_pdfs(folder_path)
text = list(pdf_texts.values())  # 論文の本文
filename = list(pdf_texts.keys())  # 対応するファイル名

In [22]:
data = []
for filename, text in pdf_texts.items():
    char_count = len(text)  # 文字数
    top_words = get_top_words(text)  # 最頻ワード20個
    conclusion_length = extract_conclusion_length(text)  # 結論の文字数
    summary = generate_summary(text)  # 要約

    data.append([filename, char_count] + top_words + [summary, conclusion_length])

In [28]:
# ---- 6. CSV保存 ----
columns = ["title", "char_count"] + [f"top_word_{i+1}" for i in range(20)] + ["summary", "conclusion_char_count"]
df = pd.DataFrame(data, columns=columns)
df.to_csv("論文データ.csv", index=False, encoding="utf-8-sig")

PermissionError: [Errno 13] Permission denied: '論文データ.csv'